In [8]:
!pip install xmlrunner

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for xmlrunner: filename=xmlrunner-1.7.7-py3-none-any.whl size=6232 sha256=07e638a381908da319eca8c115315c8e30aec7f3bec0e3cd3cab4b87ccec3d4a
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\5b\0e\9e\beb68507e2c1389e43d8e7f3df6038ede3564ab43487861644
Successfully built xmlrunner


In [13]:
# import unittest
# import pandas as pd
# from xmlrunner import XMLTestRunner

In [14]:
# Create a test file
with open("test_order_analysis.py", "w") as f:
    f.write("""
import unittest
import pandas as pd

class TestOrderAnalysis(unittest.TestCase):

    def setUp(self):
        # Load the actual dataset
        self.df = pd.read_csv('orders.csv')
        self.df['order_date'] = pd.to_datetime(self.df['order_date'])

    def test_monthly_revenue(self):
        self.df['month'] = self.df['order_date'].dt.to_period('M')
        monthly_revenue = self.df.groupby('month').apply(
            lambda x: (x['product_price'] * x['quantity']).sum()
        ).reset_index(name='total_revenue')
        self.assertEqual(
            monthly_revenue['total_revenue'].sum(),
            self.df['product_price'].mul(self.df['quantity']).sum()
        )  # Total revenue

    def test_product_revenue(self):
        product_name = self.df['product_name'].iloc[0]  # Taking the first product name for example
        product_revenue = self.df.groupby('product_name').apply(
            lambda x: (x['product_price'] * x['quantity']).sum()
        ).reset_index(name='total_revenue')
        expected_revenue = self.df[self.df['product_name'] == product_name].apply(
            lambda x: x['product_price'] * x['quantity'], axis=1
        ).sum()
        self.assertEqual(
            product_revenue.loc[product_revenue['product_name'] == product_name, 'total_revenue'].values[0],
            expected_revenue
        )

    def test_customer_revenue(self):
        customer_revenue = self.df.groupby('customer_id').apply(
            lambda x: (x['product_price'] * x['quantity']).sum()
        ).reset_index(name='total_revenue')
        customer_id = self.df['customer_id'].iloc[0]  # Taking the first customer_id for example
        expected_revenue = self.df[self.df['customer_id'] == customer_id].apply(
            lambda x: x['product_price'] * x['quantity'], axis=1
        ).sum()
        self.assertEqual(
            customer_revenue.loc[customer_revenue['customer_id'] == customer_id, 'total_revenue'].values[0],
            expected_revenue
        )

    def test_top_customers(self):
        customer_revenue = self.df.groupby('customer_id').apply(
            lambda x: (x['product_price'] * x['quantity']).sum()
        ).reset_index(name='total_revenue')
        top_customers = customer_revenue.nlargest(10, 'total_revenue')
        top_customer_id = customer_revenue.nlargest(1, 'total_revenue').iloc[0]['customer_id']
        expected_top_customer_id = self.df.groupby('customer_id').apply(
            lambda x: (x['product_price'] * x['quantity']).sum()
        ).nlargest(1).index[0]
        self.assertEqual(top_customer_id, expected_top_customer_id)

if __name__ == '__main__':
    unittest.main()
""")


In [12]:
# if __name__ == '__main__':
#     with open('test-reports.xml', 'wb') as output:
#         unittest.main(testRunner=XMLTestRunner(output=output), verbosity=2)

In [15]:
!pytest test_order_analysis.py -v --tb=short

============================= test session starts =============================
platform win32 -- Python 3.11.7, pytest-7.4.0, pluggy-1.0.0 -- C:\Users\HP\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\HP\Desktop\Tanxfi
plugins: anyio-4.2.0, Faker-9.3.1
collecting ... collected 4 items

test_order_analysis.py::TestOrderAnalysis::test_customer_revenue PASSED  [ 25%]
test_order_analysis.py::TestOrderAnalysis::test_monthly_revenue PASSED   [ 50%]
test_order_analysis.py::TestOrderAnalysis::test_product_revenue PASSED   [ 75%]
test_order_analysis.py::TestOrderAnalysis::test_top_customers PASSED     [100%]

============================== 4 passed in 2.26s ==============================
